In [ ]:
instructions = [x.strip() for x in open('input1.txt', 'r').read().split(',')]

directions = [(1,0), (0,1), (-1, 0), (0, -1)]
direction = 0

visited = set()
coordinate = (0, 0)
visited.add(coordinate)
first_double = None

for instruction in instructions:
    if instruction[0] == 'R':
        direction = (direction + 1) % len(directions)
    else:
        direction = (direction - 1) % len(directions)

    distance = int(instruction[1:])
    for i in range(1, distance+1):
        #print(coordinate)
        coordinate = (coordinate[0] + directions[direction][0], coordinate[1] + directions[direction][1])
        
        if coordinate in visited and first_double is None:
            #print(visited)
            #print(coordinate)
            first_double = coordinate

        visited.add(coordinate)

print(f'The distance to Easter Bunny headquarters is {abs(coordinate[0]) + abs(coordinate[1])}')
print(f'The distance to the first location visited twice is {abs(first_double[0]) + abs(first_double[1])}')

In [ ]:
instructions = open('input2.txt', 'r').readlines()
numpad = [['1', '2', '3'], ['4', '5', '6'], ['7', '8', '9']]
weird_numpad = [[None, None, '1', None, None], [None, '2', '3', '4', None], ['5', '6', '7', '8', '9'], [None, 'A', 'B', 'C', None], [None, None, 'D', None, None]]

x, y = 1, 1
weird_x, weird_y = 3, 3

moves = {'U': (-1, 0), 'R': (0, 1), 'D': (1, 0), 'L': (0, -1)}
def perform_move(move, x, y, weird_x, weird_y):
    dx, dy = moves[move]
    x = max(0, min(2, x+dx))
    y = max(0, min(2, y+dy))

    try:
        if weird_numpad[weird_x+dx][weird_y] is not None:
            weird_x += dx
    except IndexError:
        pass

    try:
        if weird_numpad[weird_x][weird_y+dy] is not None:
            weird_y += dy
    except IndexError:
        pass
        
    return x, y, weird_x, weird_y

code = ''
weird_code = ''

for instruction in instructions:
    for c in instruction.strip():
        x, y, weird_x, weird_y = perform_move(c, x, y, weird_x, weird_y)
    code += numpad[x][y]
    weird_code += weird_numpad[weird_x][weird_y]
print(code)
print(weird_code)

In [ ]:
triangles = [[int(x) for x in line.strip().split()] for line in open('input3.txt', 'r').readlines()]
def check_triangle(triangle):
    a, b, c = sorted(triangle)
    if a + b <= c or a + c <= b or b + c <= a:
        return False
    return True
print(f'The number of valid triangles is {sum(1 for triangle in triangles if check_triangle(triangle))}')

new_triangles = []
for i in range(0, len(triangles), 3):
    for j in range(len(triangles[0])):
        triangle = []
        triangle.append(triangles[i][j])
        triangle.append(triangles[i+1][j])
        triangle.append(triangles[i+2][j])
        new_triangles.append(triangle)

print(f'The number of valid other triangles is {sum(1 for triangle in new_triangles if check_triangle(triangle))}')

In [ ]:
import re
rooms = open('input4.txt', 'r').read().strip().split('\n')

def count_and_sort(string):
    list = []
    for c in set(string):
        if c == '-':
            continue
        list.append((c, string.count(c)))
    list.sort(key=lambda e:e[0])
    list.sort(key=lambda e:e[1], reverse=True)
    return list

valid = []

for room in rooms:
    match = re.search('(.+?)([0-9]+)\\[(.+)\\]', room)
    name, id, checksum = match.group(1), int(match.group(2)), match.group(3)
    common = count_and_sort(name)
    is_valid = True
    
    for index, c in enumerate(checksum):
        if c != common[index][0]:
            is_valid = False
            break

    if is_valid:
        valid.append((name, id))

print(f'The sum of the valid sector IDs is {sum(id for name, id in valid)}')

for encrypted, id in valid:
    decrypted = ''
    for c in encrypted:
        if c == '-':
            decrypted += ' '
            continue
        cipher = ord(c) - ord('a')
        cipher = (cipher + id) % (ord('z') - ord('a') + 1) + ord('a')
        decrypted += chr(cipher)

    if 'north' in decrypted.lower():
        print(f'The room id of the room \'{decrypted.strip()}\' is {id}')

In [ ]:
import hashlib

start_phrase = 'wtnhxymk'
password = ''
stronger_password = ['*' for i in range(8)]
accepted_positions = set()

index = 0
while len(accepted_positions) < 8:
    while not (hash := hashlib.md5((start_phrase + str(index)).encode('UTF-8')).hexdigest()).startswith('00000'):
        index += 1
    
    if len(password) < 8:
        password += hash[5]
        print(f'\rThe simple password is {password}', end='')
        if len(password) == 8:
            print()
            print(f'\rThe stronger password is {"".join(c for c in stronger_password)}', end='')

    position = int(hash[5], 16)
    if position < 8 and position not in accepted_positions:
        accepted_positions.add(position)
        stronger_password[position] = hash[6]
        if len(password) == 8:
            print(f'\rThe stronger password is {"".join(c for c in stronger_password)}', end='')
    index += 1

In [ ]:
message = open('input6.txt', 'r').read().strip().split('\n')
columns = [''.join(row[i] for row in message) for i in range(len(message[0]))]
decoded = ''.join(max([chr(c) for c in range(ord('a'), ord('z')+1)], key=lambda c: column.count(c)) for column in columns)
print(f'The decoded messages is {decoded}')
decoded = ''.join(min([chr(c) for c in range(ord('a'), ord('z')+1)], key=lambda c: column.count(c)) for column in columns)
print(f'The other decoded message is {decoded}')

In [ ]:
ips = open('input7.txt', 'r').readlines()
valid_ips = []
ssl_ips = []
for ip in ips:
    deepness = 0
    valid = False
    invalid = False
    outside = set()
    inside = set()
    for index, c in enumerate(ip):
        if c == '[':
            deepness += 1
        if c == ']':
            deepness = max(deepness-1, 0)

        if index > 2 and ip[index-3] == c and ip[index-2] == ip[index-1] and ip[index-1] != c:
            if deepness != 0:
                invalid = True
            else:
                valid = True

        if index > 1 and ip[index-2] == c:
            if deepness > 0:
                inside.add(ip[index-2:index+1])
            else:
                outside.add(ip[index-2:index+1])

    for sequence in outside:
        opposite = sequence[1] + sequence[0] + sequence[1]
        if opposite in inside:
            ssl_ips.append(ip)
            break
        
    if valid and not invalid:
        valid_ips.append(ip)
print(f'There are {len(valid_ips)} valid ips')
print(f'There are {len(ssl_ips)} valid SSL ips')

In [ ]:
import re
instructions = '''rect 3x2
rotate column x=1 by 1
rotate row y=0 by 4
rotate column x=1 by 1'''.split('\n')
instructions = open('input8.txt', 'r').readlines()
screen = [['.' for i in range(50)] for j in range(6)]


for instruction in instructions:
    if (m := re.search('rect ([0-9]+)x([0-9]+)', instruction)) is not None:
        width = int(m.group(1))
        height = int(m.group(2))
        for x in range(width):
            for y in range(height):
                screen[y][x] = '#'

    if (m := re.search('rotate row y=([0-9]+) by ([0-9]+)', instruction)) is not None:
        row_no = int(m.group(1))
        rotation = int(m.group(2))
        screen[row_no] = [screen[row_no][(i - rotation) % len(screen[row_no])] for i in range(len(screen[row_no]))]

    if (m := re.search('rotate column x=([0-9]+) by ([0-9]+)', instruction)) is not None:
        column_no = int(m.group(1))
        rotation = int(m.group(2))

        column = [screen[i][column_no] for i in range(len(screen))]
        column = [column[(i - rotation) % len(column)] for i in range(len(column))]
        for y in range(len(column)):
            screen[y][column_no] = column[y]

print(f'Number of turned on pixels: {sum(sum(1 for x in row if x == "#") for row in screen)}')
print('\n'.join(''.join(row) for row in screen))

In [ ]:
import re

compressed = open('input9.txt', 'r').read().strip()

def get_decompressed_length(string, recursive=False):
    compressed = string
    length = 0
    while (m := re.search(r'\(([0-9]+)x([0-9]+)\)', compressed)):
        start, end = m.span()
        length += start

        letters = int(m.group(1))
        repeats = int(m.group(2))

        if recursive:
            length += repeats * get_decompressed_length(compressed[end:end+letters], recursive)
        else:
            length += repeats * letters
        compressed = compressed[end+letters:]
    return length + len(compressed)

print(f'The decompressed length is {get_decompressed_length(compressed)}')
print(f'The mega-decompressed length is {get_decompressed_length(compressed, True)}')

In [ ]:
import re
instructions = open('input10.txt', 'r').readlines()

bots = {}
outputs = {}

def propagate(bot):
    if 'propagated' in bot and bot['propagated']:
        return
    
    if len(bot['inputs']) == 2 and 'high_output' in bot and 'low_output' in bot:
        high_input = max(bot['inputs'])
        low_input = min(bot['inputs'])

        if high_input == 67 and low_input == 17:
            print('EUREKA')
        
        bot['high_output']['inputs'].append(high_input)
        bot['low_output']['inputs'].append(low_input)

        propagate(bot['high_output'])
        propagate(bot['low_output'])

        bot['propagated'] = True

for index, instruction in enumerate(instructions):
    if (m := re.search('bot ([0-9]+) gives low to (bot|output) ([0-9]+) and high to (bot|output) ([0-9]+)', instruction)) is not None:
        source_bot = int(m.group(1))
        if source_bot not in bots:
            bots[source_bot] = {'inputs': []}

        if m.group(2) == 'bot':
            low_target_bot = int(m.group(3))
            if low_target_bot not in bots:
                bots[low_target_bot] = {'inputs': []}

            bots[source_bot]['low_output'] = bots[low_target_bot]
        elif m.group(2) == 'output':
            low_target_output = int(m.group(3))
            if low_target_output not in outputs:
                outputs[low_target_output] = {'inputs': []}

            bots[source_bot]['low_output'] = outputs[low_target_output]

        if m.group(4) == 'bot':
            high_target_bot = int(m.group(5))
            if high_target_bot not in bots:
                bots[high_target_bot] = {'inputs': []}

            bots[source_bot]['high_output'] = bots[high_target_bot]

        elif m.group(4) == 'output':
            high_target_output = int(m.group(5))
            if high_target_output not in outputs:
                outputs[high_target_output] = {'inputs': []}
            bots[source_bot]['high_output'] = outputs[high_target_output]
            
    elif (m := re.search('value ([0-9]+) goes to bot ([0-9]+)', instruction)) is not None:
        source_bot = int(m.group(2))
        value = int(m.group(1))
        if source_bot not in bots:
            bots[source_bot] = {'inputs': []}

        bots[source_bot]['inputs'].append(value)

    propagate(bots[source_bot])

for bot in bots:
    if 17 in bots[bot]['inputs'] and 61 in bots[bot]['inputs']:
        print(f'Bot {bot} compares 17 and 61')

multiplied = 1
for i in range(3):
    multiplied *= outputs[i]['inputs'][0]
print(f'The product of the three first input bins is {multiplied}')

In [ ]:
%run ../Common/CommonMethods.ipynb
import re
import time
import cProfile

lines = '''The first floor contains a hydrogen-compatible microchip and a lithium-compatible microchip.
The second floor contains a hydrogen generator.
The third floor contains a lithium generator.
The fourth floor contains nothing relevant.'''.strip().split('\n')

# Read the input and create a state vector
lines = open('input11.txt', 'r').read().strip().split('\n')
generators_by_floor = []
microchips_by_floor = []
for line in lines:
	generators_by_floor.append(re.findall(r'(\S+) generator', line))
	microchips_by_floor.append(re.findall(r'(\S+)-compatible microchip', line))

materials = set([material for floor in generators_by_floor for material in floor])

state_vector = tuple()
for material in materials:
	microchip_floor = next(index for index, materials in enumerate(microchips_by_floor) if material in materials)
	generator_floor = next(index for index, materials in enumerate(generators_by_floor) if material in materials)

	state_vector += (microchip_floor, generator_floor)

# Add elevator floor to end of state vector
state_vector += (0,)

def is_valid_state(state_vector):
	unmatched_microchips = set(floor for index, floor in enumerate(state_vector[:-1:2]) if state_vector[2*index+1] != floor)
	generators = set(floor for floor in state_vector[1:-1:2])
	return not bool(unmatched_microchips.intersection(generators))

def get_valid_transitions(state_vector):
	valid_transitions = []
	
	elevator_floor = state_vector[-1]
	on_same_floor = [index for index, floor in enumerate(state_vector[:-1]) if floor == elevator_floor]

	for index_one in range(len(on_same_floor)):
		for index_two in range(index_one, len(on_same_floor)):
			moved_items = set([on_same_floor[index_one], on_same_floor[index_two], len(state_vector)-1])
			if elevator_floor != 3:
				valid_transitions.append(tuple(1 if i in moved_items else 0 for i in range(len(state_vector))))
			if elevator_floor != 0:
				valid_transitions.append(tuple(-1 if i in moved_items else 0 for i in range(len(state_vector))))
	return valid_transitions

def sort_vector(state_vector):
	elevator_floor = state_vector[-1]
	pairs = [state_vector[i:i+2] for i in range(0,len(state_vector)-1,2)]
	pairs.sort(key=lambda pair: pair[0])
	state_vector = tuple()
	for a, b in pairs:
		state_vector += (a, b)
	return state_vector + (elevator_floor,)

def add_vectors(state_vector, transition_vector):
	return tuple(a+b for a, b in zip(state_vector, transition_vector))

def get_neighbours_function(current_vector):
	neighbours = set()
	for transition_vector in get_valid_transitions(current_vector):
		neighbour = sort_vector(add_vectors(current_vector, transition_vector))
		if is_valid_state(neighbour):
			neighbours.add(neighbour)
	return neighbours

def heuristic_function(current_vector):
	total_distance = 0
	for i in range(len(current_vector) // 2):
		total_distance += (3 - min(current_vector[2*i:2*i+2])) * (1 if i == 0 else 2)
	return total_distance
	

def get_number_of_steps(state_vector):
	state_vector = sort_vector(state_vector)
	before = time.time()
	goal = tuple(3 for i in range(len(state_vector)))
	path, _ = perform_a_star(state_vector, goal, get_neighbours_function, lambda current, neighbour: 1, heuristic_function)
	after = time.time()
	return path[-1][1], (after - before)

steps, time_elapsed = get_number_of_steps(state_vector)
print(f'Part one was calculated in {time_elapsed} seconds, can be done in {steps} steps')
state_vector += (0,0,0,0)
steps, time_elapsed = get_number_of_steps(state_vector)
print(f'Part two was calculated in {time_elapsed} seconds, can be done in {steps} steps')

In [ ]:
import re

instructions = '''cpy 41 a
inc a
inc a
dec a
jnz a 2
dec a'''.split('\n')
instructions = open('input12.txt', 'r').read().strip().split('\n')

def get_value(string):
	try:
		value = int(string)
	except:
		value = registers[string]
	return value

def run_program(registers):
	head = 0
	while 0 <= head < len(instructions):
		instruction = instructions[head]
		if (match := re.match(r'cpy (\S+) (\S+)', instruction)) is not None:
			registers[match.group(2)] = get_value(match.group(1))
	
		elif (match := re.match(r'inc (\S+)', instruction)) is not None:
			registers[match.group(1)] += 1
	
		elif (match := re.match(r'dec (\S+)', instruction)) is not None:
			registers[match.group(1)] -= 1
	
		elif (match := re.match(r'jnz (\S+) (\S+)', instruction)) is not None:
			if get_value(match.group(1)) != 0:
				head += int(match.group(2))
				continue
	
		head += 1
	return registers['a']

registers = {'a': 0, 'b': 0, 'c': 0, 'd': 0}
print(f'The value in register a after part one is {run_program(registers)}')
registers = {'a': 0, 'b': 0, 'c': 1, 'd': 0}
print(f'The value in register a after part two is {run_program(registers)}')

In [ ]:
%run ../Common/CommonMethods.ipynb
favourite_number = 1358

def is_wall(position):
	x, y = position
	binary = bin(x*x + 3*x + 2*x*y + y + y*y + favourite_number)
	return sum(1 for c in binary if c == '1') % 2 == 1

def find_neighbours(position):
	neighbours = []
	x, y = position
	for dx, dy in [(1,0), (0,1), (-1,0), (0,-1)]:
		nx = x+dx
		ny = y+dy

		if nx < 0 or ny < 0:
			continue
		
		if not is_wall((nx, ny)):
			neighbours.append((nx, ny))
	return neighbours

def create_heuristic_function(goal):
	gx, gy = goal
	def heuristic(current):
		# Manhattan distance
		cx, cy = current
		return abs(gx - cx) + abs(gy - cy)
	return heuristic

start = (1, 1)
goal = (31, 39)

path, _ = perform_a_star(start, goal, find_neighbours, heuristic_function=create_heuristic_function(goal))
print(f'The length of the path from {start} to {goal} is {path[-1][1]}')

# Find all reachable
_, extra = perform_a_star(start, (1e100,1e100), find_neighbours, max_search_distance=50)
print(f'The number of reachable coordinates within 50 steps is {len(extra['g_score'])}')

In [ ]:
import hashlib
from collections import defaultdict

def find_character_runs(string, run_length):
	characters = []
	for i in range(len(string)-run_length+1):
		test = string[i:i+run_length]
		if test[0]*run_length == test:
			characters.append(test[0])

	return characters

runs = defaultdict(lambda: defaultdict(lambda: list()))
stretched_runs = defaultdict(lambda: defaultdict(lambda: list()))

salt = 'qzyelonm'
counter = 0

def get_keys(runs):
	keys = []

	possible_keys = [(index, character, hash_string) for character in runs[3] for index, first, hash_string in runs[3][character] if first]
	possible_keys.sort(key=lambda t:t[0])

	for index, character, hash_string in possible_keys:
		for other_index, _, other_hash_string in runs[5][character]:
			if index < other_index <= index + 1000:
				keys.append((index, hash_string, other_index, other_hash_string))
				break
	return keys

while True:
	hash_input = salt + str(counter)
	hash_string = ''.join(f'{x:02x}' for x in hashlib.md5(hash_input.encode()).digest())

	for index, character in enumerate(find_character_runs(hash_string, 3)):
		first = index == 0
		runs[3][character].append((counter, first, hash_string))

	for index, character in enumerate(find_character_runs(hash_string, 5)):
		first = index == 0
		runs[5][character].append((counter, first, hash_string))

	for i in range(2016):
		hash_string = ''.join(f'{x:02x}' for x in hashlib.md5(hash_string.encode()).digest())

	for index, character in enumerate(find_character_runs(hash_string, 3)):
		first = index == 0
		stretched_runs[3][character].append((counter, first, hash_string))

	for index, character in enumerate(find_character_runs(hash_string, 5)):
		first = index == 0
		stretched_runs[5][character].append((counter, first, hash_string))

	counter += 1

	if counter % 1000 == 0:
		keys = get_keys(runs)
		stretched_keys = get_keys(stretched_runs)
		if len(keys) > 63 and counter > keys[63][0] + 1000 and len(stretched_keys) > 63 and counter > stretched_keys[63][0]:
			print(f'The keypad is generated after hashing index number {keys[63][0]}')
			print(f'The stretched keypad is generated after hashing index number {stretched_keys[63][0]}')
			break

In [ ]:
import re
import math
from collections import defaultdict

disc_positions = '''Disc #1 has 5 positions; at time=0, it is at position 4.
Disc #2 has 2 positions; at time=0, it is at position 1.'''.split('\n')

disc_positions = open('input15.txt').read().strip().split('\n')

discs = []
for line in disc_positions:
	match = re.match(r'.*? has (\S+) positions.*?position (\S+)\.', line)
	discs.append((int(match.group(1)), int(match.group(2))))

def factorize(number):
	factors = defaultdict(lambda: int(0))
	remaining = number
	test_factor = 2

	while test_factor <= math.sqrt(remaining):
		while remaining % test_factor == 0:
			factors[test_factor] += 1
			remaining //= test_factor
		test_factor += 1

	if remaining != 1:
		factors[remaining] += 1
	return factors
	

def find_least_common_multiple(number_one, number_two):
	factors_one = factorize(number_one)
	factors_two = factorize(number_two)

	multiple = 1

	for factor in factors_one:
		power_one = factors_one[factor]
		power_two = factors_two[factor] if factor in factors_two else 0
		multiple *= factor**max(power_one, power_two)
	for factor in factors_two:
		if factor in factors_one:
			continue
		multiple *= factor**factors_two[factor]
	return multiple


def calculate_disc_wait_time(discs):
	wait_time = 0
	increment = 1

	for index, (disc_size, disc_position) in enumerate(discs):
		target = (disc_size - index-1) % disc_size
		
		while (disc_position + wait_time) % disc_size != target:
			wait_time += increment

		increment = find_least_common_multiple(increment, disc_size)

	return wait_time

print(f'You have to wait {calculate_disc_wait_time(discs)} seconds to push the button the first time')
discs.append((11, 0))
print(f'You have to wait {calculate_disc_wait_time(discs)} seconds to push the button the second time')

In [ ]:
def dragonify(data):
	b = data[::-1]
	b = ''.join('1' if c == '0' else '0' for c in b)
	return data + '0' + b

def checksumify(data):
	if len(data) % 2 == 1:
		return data

	checksum = ''.join('1' if data[i] == data[i+1] else '0' for i in range(0,len(data),2))
	return checksum

start_string = '10010000000110000'

def find_checksum(start_string, max_length):
	dragon_string = start_string
	while len(dragon_string) < max_length:
		dragon_string = dragonify(dragon_string)
	dragon_string = dragon_string[:max_length]
	
	while len(dragon_string) % 2 == 0:
		dragon_string = checksumify(dragon_string)
	return dragon_string

print(f'The first checksum is {find_checksum(start_string, 272)}')
print(f'The second checksum is {find_checksum(start_string, 35651584)}')

In [ ]:
%run ../Common/CommonMethods.ipynb
import hashlib

passcode = 'pgflpeqp'
unlocking = set(['b', 'c', 'd', 'e', 'f'])

def goal_function(position):
	x, y, _ = position
	if x == 3 and y == 3:
		return True
	else:
		return False

def get_neighbours(position):
	neighbours = []
	
	x, y, path = position
	current_hash = hashlib.md5((passcode + path).encode()).hexdigest()

	if y > 0 and current_hash[0] in unlocking:
		neighbours.append((x, y-1, path + 'U'))

	if y < 3 and current_hash[1] in unlocking:
		neighbours.append((x, y+1, path + 'D'))

	if x > 0 and current_hash[2] in unlocking:
		neighbours.append((x-1, y, path + 'L'))

	if x < 3 and current_hash[3] in unlocking:
		neighbours.append((x+1, y, path + 'R'))

	return neighbours

def heuristic(position):
	x, y, _ = position
	return abs(3 - x) + abs(3 - y)
	
best_path = perform_a_star((0,0,''), goal_function, get_neighbours, heuristic_function=heuristic)
print(f'The best path is {best_path[0][-1][0][2]}')

def negative_heuristic(position):
	x, y, path = position

	if goal_function(position):
		return 1e10 - len(path)
	return -(abs(3 - x) + abs(3 - y) + len(path))

best_path = perform_a_star((0,0,''), goal_function, get_neighbours, heuristic_function=negative_heuristic, distance_function=lambda current, neighbour: 0)
print(f'The length of the longest possible path is {len(best_path[0][-1][0][2])}')

In [ ]:
starting_row = open('input18.txt', 'r').read().strip()

row_length = len(starting_row)

row = 0
traps = set([(index, row) for index, floor_type in enumerate(starting_row) if floor_type == '^'])

def add_row(row):
	for column in range(row_length):
		left = (column - 1, row - 1) in traps
		center = (column, row - 1) in traps
		right = (column + 1, row - 1) in traps

		if (
			(left and center and not right)
			or (not left and center and right)
			or (left and not center and not right)
			or (not left and not center and right)
		):
			traps.add((column, row))

while row < 39:
	row += 1
	add_row(row)

print(f'There are {(row+1) * row_length - len(traps)} safe squares')

while row < 399999:
	row += 1
	add_row(row)
print(f'There are {(row+1) * row_length - len(traps)} safe squares')

In [ ]:
import math

# Josephus problem
number_of_elves = 3014387
power_of_two = math.floor(math.log(number_of_elves)/math.log(2))
l = number_of_elves - 2**power_of_two
print(f'The survivor is elf number {2*l + 1}')

# Brute force part two
sequence = []
while number_of_elves > 1:
	sequence.append(number_of_elves)
	number_of_elves = math.ceil(number_of_elves / 3)

survivor = 1
sequence.pop()
for number in sequence[::-1]:
	if number % 3 == 0:
		survivor *= 3
	elif number % 3 == 1:
		survivor = 3 * (survivor - 1) + 1
	else:
		survivor = 3 * survivor - 1
print(f'The survivor of the second game is elf number {survivor}')

In [ ]:
blocked_ranges = open('input20.txt', 'r').read().strip().split('\n')

def check_if_ranges_intersect(range_one, range_two):
	l1, r1 = range_one
	l2, r2 = range_two
	if l1 <= l2 <= r1 or l1 <= r2 <= r1 or l1 <= l2 <= r2 <= l2 or l2 <= l1 <= r1 <= r2:
		return True
	else:
		return False

ranges = set()
for blocked_range in blocked_ranges:
	current_range = tuple(int(x) for x in blocked_range.split('-'))
	finished_comparing = False
	while not finished_comparing:
		combined = False
		for other_range in ranges:
			if check_if_ranges_intersect(current_range, other_range):
				combined = True
				ranges.remove(other_range)
				current_range = (min(current_range[0], other_range[0]), max(current_range[1], other_range[1]))
				break
		if not combined:
			finished_comparing = True
	ranges.add(current_range)

ranges = list(sorted(ranges, key=lambda t:t[0]))

counter = 0
while counter < len(ranges):
	first_range = ranges[0]
	second_range = ranges[1]

	if first_range[1] + 1 == second_range[0]:
		del ranges[1]
		del ranges[0]

		ranges.insert(0,(first_range[0], second_range[1]))
	else:
		counter += 1

print(f'The first allowed IP is {ranges[0][1]+1}')
print(f'The number of allowed IPs is {sum(ranges[i][0] - ranges[i-1][1] - 1 for i in range(1,len(ranges)))}')